**COERENZA LSA**



**1 - IMPORT AND PREPROCESSING**



In [ ]:
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

import pandas as pd
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim
from gensim.utils import simple_preprocess

from google.colab import drive 
drive.mount('/content/drive')

import re
import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/drive


In [ ]:
filtered_dataset = pd.read_csv('/content/drive/MyDrive/PYTHON/Latent Dirichlet Alg/dataset_informatica_full.csv')
filtered_dataset = filtered_dataset.drop(columns = ['keywords', 'Unnamed: 0', 'Unnamed: 0.1','authors', 'language',
                                'difficulty', 'format', 'duration', 'public', 'min_age', 'max_age',
                                'resource_image', 'WikiLink','link','type', 'Upload_Date'], axis=1)
filtered_dataset = filtered_dataset[filtered_dataset['WikiDescription']!="0"]
filtered_dataset = filtered_dataset[filtered_dataset['WikiDescription']!="-"]
filtered_dataset = filtered_dataset[filtered_dataset['WikiDescription']!='']

filtered_dataset['text_preprocessed']=filtered_dataset['WikiDescription'].map(lambda x: re.sub('[,.\!?]', '', x))
filtered_dataset['text_preprocessed']=filtered_dataset['text_preprocessed'].map(lambda x: x.lower())

filtered_dataset[:2]

,disciplines,title,description,WikiDescription,text_preprocessed
0,Applied Science/Computer Science,Intro to Scratch,"In this lesson, students will begin exploring ...",Scratch è un ambiente di programmazione gratui...,scratch è un ambiente di programmazione gratui...
1,Applied Science/Computer Science,Intro to Scratch,Students encounter the Scratch programming pla...,"Scratch è adatto a studenti, insegnanti e geni...",scratch è adatto a studenti insegnanti e genit...


In [ ]:
stop_words = stopwords.words('italian')

def sent_to_words(sentence):
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) #converting the raw text into a list of tokens (words) deacc removes accents
    
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

In [ ]:
for index, row in filtered_dataset.iterrows():
        data = row['text_preprocessed']
        if (data != '' or data != '-') :
            words = sent_to_words(data) #da scrivere list
            words = remove_stopwords(words)
            row['text_preprocessed']=str(words)


In [ ]:
filtered_dataset[:2]

,disciplines,title,description,WikiDescription,text_preprocessed
0,Applied Science/Computer Science,Intro to Scratch,"In this lesson, students will begin exploring ...",Scratch è un ambiente di programmazione gratui...,"[['scratch', 'ambiente', 'programmazione', 'gr..."
1,Applied Science/Computer Science,Intro to Scratch,Students encounter the Scratch programming pla...,"Scratch è adatto a studenti, insegnanti e geni...","[['scratch', 'adatto', 'studenti', 'insegnanti..."


In [ ]:
riga = filtered_dataset['text_preprocessed'][1]

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation \
                                        , preprocess_string, strip_short, stem_text

# preprocess given text
def preprocess(text):
    
    # clean text based on given filters
    CUSTOM_FILTERS = [lambda x: x.lower(),
                                strip_punctuation, 
                                #strip_short,
                                ]

    text = preprocess_string(text, CUSTOM_FILTERS)
    
    return text

# apply function to all reviews 
filtered_dataset['text_preprocessed'] = filtered_dataset['text_preprocessed'].apply(lambda x: preprocess(x))

In [ ]:
filtered_dataset[:2]

,disciplines,title,description,WikiDescription,text_preprocessed
0,Applied Science/Computer Science,Intro to Scratch,"In this lesson, students will begin exploring ...",Scratch è un ambiente di programmazione gratui...,"[scratch, ambiente, programmazione, gratuito, ..."
1,Applied Science/Computer Science,Intro to Scratch,Students encounter the Scratch programming pla...,"Scratch è adatto a studenti, insegnanti e geni...","[scratch, adatto, studenti, insegnanti, genito..."


In [ ]:
filtered_dataset['text_preprocessed'][1]

'scratch'

**IMPLEMENTAZIONE LSA**

In [ ]:
### PROVA

from gensim import corpora
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

def lsa_model(dataset, num_topics):
  
  corpus = dataset['text_preprocessed']
  dictionary = corpora.Dictionary(corpus) 
  bow = [dictionary.doc2bow(text) for text in corpus]

  lsi = LsiModel(bow, num_topics=num_topics, id2word=dictionary)
  
  coherence_model = CoherenceModel(model=lsi,
                                   texts = dataset['text_preprocessed'],
                                   dictionary = dictionary, coherence='c_v')
  coherence_score = coherence_model.get_coherence()

  return coherence_score

In [ ]:
## prova

c_list = []
index = []

for i in range(1,15):
  c = lsa_model(filtered_dataset, num_topics = i)
  c_list.append(c)
  index.append(i)

In [ ]:
## prova

coh_dict = {'num_topics':index,
            'coherence':c_list
            }

coh_df = pd.DataFrame(coh_dict)
coh_df
#coh_df.sort_values(by = 'coherence', ascending = False)

In [ ]:
#### FUNZIONE FINALE

import tqdm
pbar = tqdm.tqdm(total=int(filtered_dataset.shape[0]))
coherences = []
topics = []

def coherence_model(dataset, num_of_topics = 1):

  for index, row in dataset.iterrows():

    corpus = row['text_preprocessed']
    words = list(sent_to_words(corpus))
    dictionary = corpora.Dictionary(words)
    texts = words
    bow = [dictionary.doc2bow(text) for text in texts]

    lsi = LsiModel(bow, num_topics=num_of_topics, id2word=dictionary)
    topic = lsi.show_topics(num_topics = 1)
    topics.append(topic)

    coherence_model = CoherenceModel(model=lsi, texts = words,
                                     dictionary = dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    coherences.append(coherence_score)

    pbar.update(1)


  0%|          | 0/1066 [00:00<?, ?it/s]

In [ ]:
coherence_model(filtered_dataset)

100%|██████████| 1066/1066 [01:10<00:00, 17.36it/s]

In [ ]:
filtered_dataset['topics'] = topics
filtered_dataset['coherence'] = coherences
filtered_dataset.coherence.mean(), filtered_dataset.coherence.std()

(0.9603719394706481, 0.14562254597732857)

In [ ]:
filtered_dataset = filtered_dataset.reset_index()
for index, row in filtered_dataset.iterrows():
    #print(index)
    topicstringtemp = ''
    for i in range(1):
        topicstring = re.sub('[1234567890.*" ]', '', topics[index][i][1])
        topicstringtemp = topicstringtemp + topicstring
    filtered_dataset.at[index, 'topics'] = topicstringtemp

In [ ]:
filtered_dataset.head()

,level_0,index,disciplines,title,description,WikiDescription,text_preprocessed,topics,coherence
0,0,0,Applied Science/Computer Science,Intro to Scratch,"In this lesson, students will begin exploring ...",Scratch è un ambiente di programmazione gratui...,"[scratch, ambiente, programmazione, gratuito, ...",programmazione+ambiente+grafico+gratuito+lingu...,1.0
1,1,1,Applied Science/Computer Science,Intro to Scratch,Students encounter the Scratch programming pla...,"Scratch è adatto a studenti, insegnanti e geni...","[scratch, adatto, studenti, insegnanti, genito...",adatto+genitori+animazioni+matematica+spaziano...,1.0
2,2,2,Applied Science/Computer Science,Intro to Vectors Physics and Augmented Reality,Students learn about video motion capture tech...,"Per realtà aumentata, o realtà mediata dall'el...","[realta, aumentata, realta, mediata, elaborato...",realta+arricchimento+informazioni+intende+uman...,1.0
3,3,3,Applied Science/Computer Science,Intro to computers,Introduction to Computers - brief videos,"Un computer, è una macchina automatizzata prog...","[computer, macchina, automatizzata, programmab...",tipi+programmabile+eseguire+matematici+macchin...,1.0
4,4,4,Applied Science/Computer Science,Intro to the Zotero API,"In this lesson, you’ll learn how to use python...",Zotero è un software per la gestione di riferi...,"[zotero, software, gestione, riferimenti, bibl...",bibliografici+gestione+materiali+riferimenti+s...,1.0


In [ ]:
###############################################################################################

In [ ]:
from gensim import corpora


# create a dictionary with the corpus
corpus = filtered_dataset['text_preprocessed']
dictionary = corpora.Dictionary(corpus)

# convert corpus into a bag of words
bow = [dictionary.doc2bow(text) for text in corpus]

In [ ]:
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

# find the coherence score with a different number of topics
for i in range(1,15):
    lsi = LsiModel(bow, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=filtered_dataset['text_preprocessed'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 1 clusters: 0.6355236653364047
Coherence score with 2 clusters: 0.4985176122789239
Coherence score with 3 clusters: 0.7158417664086908
Coherence score with 4 clusters: 0.45559119134964576
Coherence score with 5 clusters: 0.5466763026474556
Coherence score with 6 clusters: 0.5800144189406595
Coherence score with 7 clusters: 0.607061742804509
Coherence score with 8 clusters: 0.487045974960413
Coherence score with 9 clusters: 0.47746204452121777
Coherence score with 10 clusters: 0.4843566304944037
Coherence score with 11 clusters: 0.4846675492462574
Coherence score with 12 clusters: 0.4843826227770996
Coherence score with 13 clusters: 0.448799480926423
Coherence score with 14 clusters: 0.5039861462568233


In [ ]:
lsi = LsiModel(bow, num_topics=7, id2word=dictionary)

In [ ]:
# find the 5 words with the srongest association to the derived topics
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.534*"ciclo" + 0.250*"puo" + 0.244*"valore" + 0.238*"contatore" + 0.213*"numero" + 0.206*"for" + 0.183*"essere" + 0.182*"variabile" + 0.147*"linguaggi" + 0.144*"genere".
Words in 1: 0.436*"dati" + 0.419*"web" + 0.268*"piu" + 0.227*"sviluppo" + 0.171*"software" + -0.162*"ciclo" + 0.155*"informatica" + 0.126*"programmazione" + 0.104*"rete" + 0.098*"apprendimento".
Words in 2: -0.613*"web" + 0.524*"dati" + -0.283*"sviluppo" + 0.117*"analisi" + 0.096*"apprendimento" + -0.090*"end" + -0.088*"sviluppatori" + -0.087*"sviluppatore" + 0.087*"data" + -0.081*"contenuti".
Words in 3: 0.494*"dati" + -0.343*"software" + -0.254*"programmazione" + 0.226*"web" + -0.218*"programma" + -0.176*"piu" + -0.163*"eventi" + 0.134*"analisi" + -0.131*"codice" + -0.123*"informatica".
Words in 4: 0.330*"stati" + 0.313*"apprendimento" + 0.273*"uniti" + -0.248*"software" + -0.204*"dati" + 0.202*"automatico" + -0.157*"eventi" + 0.145*"guerra" + 0.117*"riconoscimento" + 0.116*"algoritmi".
Words in 5: -0.36

In [ ]:
# find the scores given between the review and each topic
corpus_lsi = lsi[bow]
score1 = []
score2 = []
score3 = []
score4 = []
score5 = []
score6 = []
score7 = []
for doc in corpus_lsi:
    score1.append(round(doc[0][1],2))
    score2.append(round(doc[1][1],2))
    score3.append(round(doc[2][1],2))
    score4.append(round(doc[3][1],2))
    score5.append(round(doc[4][1],2))
    score6.append(round(doc[5][1],2))
    score7.append(round(doc[6][1],2))

# create data frame that shows scores assigned for both topics for each review
df_topic = pd.DataFrame()
df_topic['Text'] = filtered_dataset['text_preprocessed']
df_topic['Topic 0 score'] = score1
df_topic['Topic 1 score'] = score2
df_topic['Topic 2 score'] = score3
df_topic['Topic 3 score'] = score4
df_topic['Topic 4 score'] = score5
df_topic['Topic 5 score'] = score6
df_topic['Topic 6 score'] = score7

df_topic['Topic']= df_topic[['Topic 0 score', 'Topic 1 score', 'Topic 2 score', 'Topic 3 score', 'Topic 4 score', 'Topic 5 score', 'Topic 6 score']].apply(lambda x: x.argmax(), axis=1)
df_topic.head()

,Text,Topic 0 score,Topic 1 score,Topic 2 score,Topic 3 score,Topic 4 score,Topic 5 score,Topic 6 score,Topic
0,"[scratch, ambiente, programmazione, gratuito, ...",0.39,0.39,0.17,-0.68,-0.06,0.25,-0.59,0
1,"[scratch, adatto, studenti, insegnanti, genito...",0.05,0.21,0.12,-0.22,0.19,0.33,-0.16,5
2,"[realta, aumentata, realta, mediata, elaborato...",0.16,0.06,0.04,0.04,0.01,-0.04,0.07,0
3,"[computer, macchina, automatizzata, programmab...",0.18,0.67,0.65,0.42,-0.09,-0.16,0.16,1
4,"[zotero, software, gestione, riferimenti, bibl...",0.07,0.22,0.03,-0.35,-0.26,-0.10,0.48,6


In [ ]:
df_topic0 = df_topic[df_topic['Topic'] == 0]
df_topic1 = df_topic[df_topic['Topic']==1]
print('Sample text from topic 0:\n {}'.format(df_topic0.sample(1, random_state=2)['Text'].values))
print('\nSample text from topic 1:\n {}'.format(df_topic1.sample(1, random_state=2)['Text'].values))

Sample text from topic 0:
 [list(['python', 'linguaggio', 'programmazione', 'alto', 'livello', 'orientato', 'oggetti', 'adatto', 'altri', 'usi', 'sviluppare', 'applicazioni', 'distribuite', 'scripting', 'computazione', 'numerica', 'system', 'testing'])]

Sample text from topic 1:
 [list(['carpentries', 'organizzazione', 'senza', 'scopo', 'lucro', 'insegna', 'competenze', 'ingegneria', 'software', 'scienza', 'dati', 'ricercatori', 'attraverso', 'laboratori', 'didattici', 'carpentries', 'compone', 'tre', 'aree', 'programma', 'software', 'carpentry', 'data', 'carpentry', 'library', 'carpentry', 'workshop', 'carpentries', 'stati', 'organizzati', 'livello', 'internazionale', 'workshop', 'presso', 'smithsonian', 'institution', 'australian', 'research', 'data', 'commons', 'cern', 'antartide'])]
